# Tarefa 01: SQL

Um sistema de pagamentos, registra as transações dos seus usuários em duas tabelas:

Users: Contém o registro de usuários do sistema. Os seus atributos são:

-   id: Identificador único de usuário
-   age (idade): em anos
-   país de residência: país onde o usuários indicou que reside

Transactions: Que contém o registro de todas as transações que os usuários realizam através do sistema.

-   transaction_id: Identificador único da transação
-   transaction_date: Data na qual a transação foi realizada. O tipo é timestamp, ou seja, é um tipo de data que contém o ano, mês, dia, hora, minuto e segundos.
-   user_id: Identificador do usuário que realizou a transação (é uma chave externa à tabela users)
-   transaction_state: Campo que indica o estado da transação que pode ser *INITIATED*, *SUCCESS*, *FRAUD* or *CANCELLED*.
-   transaction_amount: Valor da transação em USD

Com estes dados disponíveis, precisamos gerar queries SQL para responder às seguintes perguntas:

*Para fins didáticos, criei esses bancos de dados para simular a resolução das questões.*


```{r}
tabela_transactions <- tibble::tribble(
                         ~user_id, ~transaction_date, ~transaction_id, ~transaction_state, ~transaction_amount,
                         "1", "2022-01-12", "255", "sucesso", "119.65",
                         "2", "2022-02-23", "585", "INITIATED", "22.63",
                         "3", "2022-04-14", "178", "CANCELLED", "225.87",
                         "4", "2022-01-12", "887", "SUCCESS", "16.58",
                         "5", "2022-08-30", "569", "FRAUD", "1225.89",
                         "6", "2022-05-25", "789", "SUCCESS", "59.90",
                         ) %>% 
                      dplyr::mutate(transaction_amount = as.numeric(transaction_amount))

tabela_user <- tibble::tribble(
                         ~id, ~age, ~pais, 
                         "1", "22", "Brasil",
                         "2", "29", "Estados Unidos",
                         "3", "24", "África do Sul",
                         "4", "32", "Bolívia",
                         "5", "28", "Brasil",
                         "6", "42", "Brasil",
                         )
```


::: panel-tabset
# A

Qual é a idade média de usuários do sistema por país?


```{r}
library(dbplyr)
library(dplyr)

con <- DBI::dbConnect(RSQLite::SQLite(), ":memory:")
copy_to(con, tabela_user)
copy_to(con, tabela_transactions)

tabela_user <- tbl(con, "tabela_user") # <1>
tabela_transactions <- tbl(con, "tabela_transactions") # <2>
```


1.  Conectando a base de dados {tabela_user} com o SQLite
2.  Conectando a base de dados {tabela_transactions} com o SQLite


```{r}
mean_tabela_user <- tabela_user %>%
                     dplyr::group_by(pais) %>% 
                     dplyr::summarise(media_idade = round(mean(age, na.rm = TRUE),2))

mean_tabela_user %>% 
  show_query()
```


::: callout-tip
# Resposta

<SQL> SELECT `pais`, ROUND(AVG(`age`), 2) AS `media_idade` FROM `tabela_user` GROUP BY `pais`
:::

# B

Qual é o país com a maior quantidade de dinheiro transacionado (considere só transações finalizadas com sucesso ou "SUCCESS")?


```{r}
tabela_merge <- tabela_transactions %>% 
                  dplyr::filter(transaction_state == "SUCCESS" | transaction_state == "sucesso") %>% 
                  dplyr::left_join(tabela_user, by = c("user_id" = "id")) %>% 
                  dplyr::slice_max(transaction_amount)

tabela_merge %>% 
  show_query()
```


::: callout-tip
# Resposta

<SQL> SELECT `user_id`, `transaction_date`, `transaction_id`, `transaction_state`, `transaction_amount`, `age`, `pais` FROM ( SELECT `q01`.*, RANK() OVER (ORDER BY `transaction_amount` DESC) AS `col01` FROM ( SELECT `LHS`.*, `age`, `pais` FROM ( SELECT `tabela_transactions`.\* FROM `tabela_transactions` WHERE (`transaction_state` = 'SUCCESS' OR `transaction_state` = 'sucesso') ) AS `LHS` LEFT JOIN `tabela_user` ON (`LHS`.`user_id` = `tabela_user`.`id`) ) AS `q01` ) AS `q01` WHERE (`col01` \<= 1)
:::

# C

Qual é o país com maior taxa de fraude em porcentagem respeito ao número de transações totais no país

# D

Na mesma linha da pergunta anterior, responda qual é a faixa de idade de usuários que mais cometem fraude (em percentagem). Separe as faixas etárias em: (\<18 anos, 18-30 anos, 30 - 45 anos, 45 - 60 anos, 60 \> anos) e considere que para responder essa pergunta, você deverá considerar o fato que um usuário pode ter executado várias transações, das quais poucas (ou muitas) podem ter sido fraude entre as demais.

# E

Imagine que a camada executiva da empresa dona do sistema, precisa criar um Dashboard para monitorar o estado das transações nos últimos 3 dias. Para isso você precisa criar uma query SQL que calcule o número e dinheiro das transações não finalizadas, número e dinheiro de transações finalizadas com sucesso (SUCCESS), o número e dinheiro de transações canceladas (CANCELLED), o número e dinheiro de fraudes (FRAUD), agrupado por país, nos 3 dias anteriores de quando o executivo da empresa consulte seu Dashboard.
:::

# Tarefa 02: Python

Numa tribo ancestral e muito desenvolvida, orientada principalmente por uma cultura lógica e matemática, os nativos estão interessados no estudo das linguagens de outras tribos e civilizações.

Durante seus estudos, estes perceberam que as linguagens dessas outras civilizações são compostas por um conjunto de símbolos (letras do alfabeto) que são agrupados e combinados para representar conceitos (palavras). Assim mesmo, estes também sabem que cada civilização possui um alfabeto específico.

Na tentativa da tribo de avançar nos seus estudos, estes desejam saber qual seria o número de palavras possíveis a serem criadas em função do tamanho da palavra e o conjunto de símbolos (alfabeto) da civilização em estudo, independentemente se essas "palavras" representam algum significado ou não.

::: panel-tabset
# A

Você, que compartilha os atributos da tribo enquanto as capacidades analíticas e lógicas, precisa ajudá-la escrevendo um algoritmo para fazer o cálculo descrito acima, considerando que tribo oferecerá para você o alfabeto da civilização e o tamanho da palavra/combinação. Seu algoritmo deveria ser capaz de calcular que o número total de palavras possíveis (independente se tem significado ou não) é de 16.

OBS: NÃO PRECISA ESCREVER A LISTA DAS PALAVRAS, A TRIBO SÓ PRECISA DO NÚMERO!

# B

Para ajudar a tribo ainda mais! Você deverá modificar seu algoritmo (ou talvez escrever um novo) para fazer o mesmo cálculo, só que agora, as palavras não podem ter símbolos repetidos.
:::

# Tarefa 03: Estatística e Machine Learning

::: panel-tabset
# 1

Dado um dataset das estaturas (em centímetros) de 13 indivíduos:

Dataset = {175, 166, 183, 193, 155, 177, 173, 171, 162, 185, 176, 161, 188}

Calcule a média, a mediana, 20 percentil, 80 percentil e desvio padrão. Considerando os dados, responda a que distribuição (paramétrica) os dados se aproximam.

# 2

Com suas próprias palavras, explique em que consiste o teorema central do limite, e se possível, mencione a sua importância no campo da inferência estatística.

# 3

Suponha que numa escola, 2 grupos diferentes de estudantes (Grupo A e Grupo B) fazem o mesmo teste de matemáticas. As pontuações para cada grupo são dados pelos seguintes datasets:

Grupo A: {80, 85, 88, 90, 92, 75, 78} Grupo B: {75, 78, 82, 85, 87, 93, 99}

Elabore um teste de hipótese para determinar se existe uma diferença estatisticamente significativa entre a média das pontuações dos dois grupos com uma confiança de 95%.

# 4

Determine a hipótese nula (Ho) e hipótese alternativa (Ha) do seguinte cenário: Uma empresa afirma que o tempo médio dos seus produtos é menos de 4 dias. Você, conta com uma amostra dessas entregas para validar estadísticamente essa afirmação.

# 5

Calcule o coeficiente de correlação de pearson das variáveis "Número de aparições Zendaya em filmes por ano" e "Número de pessoas afogadas em piscinas no Brasil ao ano". Em base nos seus resultados, considera que a variável "Número de aparições Zendaya em filmes por ano" é um bom preditor do número de pessoas afogadas no Brasil?Justifique a sua resposta.

# 6

Explique a diferença entre amostragem estratificada e amostragem randômica ou aleatória. Discuta quais são as vantagens e desvantagens de cada uma e de exemplos de casos onde uma abordagem é mais adequada que a outra.

# 7

Se você treina um modelo de Machine Learning (ou estadístico), como você identificaria se seu modelo tem uma alta variância (overfitting) ou um alto viés (bias, ou underfitting). Caso seu modelo apresenta alta variância, como você resolveria esse problema?

# 8

Considere o seguinte modelo de regressão:

-   Salário = 1200+ 500.Idade + 600. Têm Faculdade + 50.Têm Linkedin

Interprete o efeito da Idade na variável salário. Que acontece com as pessoas que não tem faculdade? Considere agora que o desvio padrão do coeficiente da variável "Têm Linkedin" é de 47, por tanto o seu p-valor é \~0.92 (como visto na equação abaixo). O que isso implica para o modelo em questão? Essa variável é relevante?
:::


```{r}
#| warning: false
#| message: false

library(dados)
library(reactable)
library(dplyr)

dados_gapminder |>                                  # <1> 
  select(continente, pais, ano, pib_per_capita) |>  # <2> 
  reactable(
    groupBy = "continente",                         # <3>
    searchable = TRUE,                              # <4>
    outlined = TRUE,                                # <5>
    columns = list(ano = colDef(sortable = TRUE)))   # <6>
```


1.  Base do pacote {dados}
2.  Selecionando algumas colunas
3.  Agrupando pela coluna 'continente'
4.  Inserindo uma busca na tabela
5.  Linhas na tabela
6.  Ordem crescente na coluna 'ano'